In [ ]:
import numpy as np
import os, os.path
from keras.preprocessing import image
import keras
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import classification_report
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Preparing the Training and Testing Dataset

In [ ]:
labels = ['paper', 'rock', 'scissors']

In [ ]:
#Inputting the path where Training images are stored
path = input("Enter the path where your training images are stored: ")

In [ ]:
#Creating a training dataset
Images = []
for label in labels:
    Path = os.path.join(path, label)
    Class = labels.index(label)
    for img in os.listdir(Path):
        Image = image.load_img(os.path.join(Path, img), target_size=(300,200,3)) #Importing the image
        Image = image.img_to_array(Image) #Converting the image into an array
        Image = Image/255.0 #Normalizing the image
        Images.append([Image, Class])
Training_Data = np.array(Images, dtype=object)      

In [ ]:
TrainingImages = []
for i in Training_Data:
    if(i[1] == 0):
        TrainingImages.append("Paper")
    elif (i[1] == 1):
        TrainingImages.append("Rock")
    else:
        TrainingImages.append("Scissors")
sns.countplot(TrainingImages).set(title='Training Data')

In [ ]:
#Inputting the path where Testing images are stored
path = input("Enter the path where your test images are stored: ")

In [ ]:
#Creating a testing dataset
Images = []
for label in labels:
    Path = os.path.join(path, label)
    Class = labels.index(label)
    for img in os.listdir(Path):
        Image = image.load_img(os.path.join(Path, img), target_size=(300,200,3)) #Importing the image
        Image = image.img_to_array(Image) #Converting the images into an array
        Image = Image/255.0 #Normalizing the image
        Images.append([Image, Class])
Testing_Data = np.array(Images, dtype=object)       

In [ ]:
TestingImages = []
for i in Testing_Data:
    if(i[1] == 0):
        TestingImages.append("Paper")
    elif (i[1] == 1):
        TestingImages.append("Rock")
    else:
        TestingImages.append("Scissors")
sns.countplot(TestingImages).set(title='Testing Data')

In [ ]:
X_train = []
Y_train = []
X_test = []
Y_test = []

for feature, label in Training_Data:
    X_train.append(feature)
    Y_train.append(label)

for feature, label in Testing_Data:
    X_test.append(feature)
    Y_test.append(label)

X_train = np.array(X_train)
X_test = np.array(X_test) 
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [ ]:
#One hot encoding the target values for the training and testing set
Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test = tf.keras.utils.to_categorical(Y_test)

# Building, Training and Testing the Model

In [ ]:
#Defining the Model
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (7, 7), activation='relu', input_shape=(300, 200, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (7, 7), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(3, activation="softmax"))
model.summary()

In [ ]:
#Compliling the Model 
LR = 0.01
opt = SGD(learning_rate=LR)
model.compile(optimizer = opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Training the Model
history = model.fit(X_train, Y_train, epochs=9, shuffle=True, validation_data=(X_test, Y_test))

In [ ]:
#Plotting the training accuracy and the validation accuracy for the Model
plt.title('Training Accuracy and Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(history.history['accuracy'], color='red', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], color='blue', label = 'Test Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
#Plotting the training loss and the validation loss for the Model
plt.title('Training Loss and Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(history.history['loss'], color='red', label='Training Loss')
plt.plot(history.history['val_loss'], color='blue', label = 'Test Loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
#Testing the Model 
loss, acc = model.evaluate(X_test, Y_test, verbose=0)
acc = round(acc,3)
print("Test Loss = ",round(loss,3))
print("Test Accuracy = ",acc * 100.0,"%")

In [ ]:
#Confusion Matrix for the Model
Y_pred = model.predict(X_test) 
Y_pred = np.argmax(Y_pred, axis=1)
Y_t = np.argmax(Y_test, axis=1)
CM = confusion_matrix(Y_t, Y_pred)

In [ ]:
#Plotting the Confuion Matrix for the Model
plt.figure(figsize=(4,4))
sns.heatmap(CM, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(acc*100.0)
plt.title(all_sample_title, size = 16);

In [ ]:
#Printing the classification report for the Model
print(classification_report(Y_t, Y_pred))

In [ ]:
#Testing accuracy for all the three classes for the Model
acc = CM[0][0]/186
acc = round(acc, 3)
print("Test Accuracy for Class Paper = ", acc * 100,"%")
acc = CM[1][1]/188
acc = round(acc, 3)
print("Test Accuracy for Class Rock = ", acc * 100,"%")
acc = CM[2][2]/188
acc = round(acc, 3)
print("Test Accuracy for Class Scissors = ", acc * 100,"%")

# Playing the Game

In [ ]:
#Inputting the path where Validation images are stored
path = input("Enter the path where your Validation images are stored: ")

In [ ]:
def play_game():
    player1 = int(input("Player 1, please enter your choice, press 0 for Paper, 1 for Rock or 2 for Scissors: "))
    if player1 == 0:
        label1 = 'paper'
        print("Player 1 picked Paper.")
    elif player1 == 1:
        label1 = 'rock'
        print("Player 1 picked Rock.")
    elif player1 == 2:
        label1 = 'scissors'
        print("Player 1 picked Scissors.")
    else:
        raise Exception("Sorry wrong input, please press between 0, 1 or 2.")
    Path = os.path.join(path, label1)
    r = random.randint(1,10) #Choosing a random integer between 1 and 10 for selecting an image 
    Image = image.load_img(os.path.join(Path, '{}.png'.format(r)), target_size=(300,200,3)) #Importing the image
    plt.figure(figsize=(3,3))
    plt.imshow(Image)
    plt.show()
    Image = image.img_to_array(Image) #Converting the image into an array
    Image = Image.reshape(1, 300, 200, 3)
    Image = Image/255.0 #Normalization
    result1 = model.predict(Image) #Predicting the image
    class1 = np.argmax(result1, axis = 1)
    
    
    player2 = int(input("Player 2, please enter your choice, press 0 for Paper, 1 for Rock or 2 for Scissors: "))
    if player2 == 0:
        label2 = 'paper'
        print("Player 2 picked Paper.")
    elif player2 == 1:
        label2 = 'rock'
        print("Player 2 picked Rock.")
    elif player2 == 2:
        label2 = 'scissors'
        print("Player 2 picked Scissors.")
    else:
        raise Exception("Sorry wrong input, please press between 0, 1 or 2.")
    Path = os.path.join(path, label2)
    r = random.randint(1,10) #Choosing a random integer between 1 and 10 for selecting an image 
    Image = image.load_img(os.path.join(Path, '{}.png'.format(r)), target_size=(300,200,3)) #Importing the image
    plt.figure(figsize=(3,3))
    plt.imshow(Image)
    plt.show()
    Image = image.img_to_array(Image) #Converting the image into an array
    Image = Image.reshape(1, 300, 200, 3)
    Image = Image/255.0 #Normalization
    result2 = model.predict(Image) #Predicting the image
    class2 = np.argmax(result2, axis = 1)
    
    if (class1 == class2):
        print("Game is a Draw.")
    elif (class1 == 0 and class2 == 1):
        print("Player 1 Wins. Since, Paper beats Rock.")
    elif (class1 == 0 and class2 == 2):
        print("Player 2 Wins. Since, Scissors beats Paper.")
    elif (class1 == 1 and class2 == 0):
        print("Player 2 Wins. Since, Paper beats Rock.")
    elif (class1 == 1 and class2 == 2):
        print("Player 1 Wins. Since, Rock beats Scissors.")
    elif (class1 == 2 and class2 == 0):
        print("Player 1 Wins. Since, Scissors beats Paper.")
    elif (class1 == 2 and class2 == 1):
        print("Player 2 Wins. Since, Rock beats Scissors")
    else:
        print("Something is Wrong.")

# Playing the game on the validation images

In [ ]:
print("Game 1:")
play_game()

In [ ]:
print("Game 2:")
play_game()

In [ ]:
print("Game 3:")
play_game()

In [ ]:
print("Game 4:")
play_game()

In [ ]:
print("Game 5:")
play_game()

In [ ]:
#Inputting the path where my own images are stored
path = input("Enter the path where your own images are stored: ")

# Playing the game on my own images

In [ ]:
print("Game 1:")
play_game()

In [ ]:
print("Game 2:")
play_game()

In [ ]:
print("Game 3:")
play_game()

In [ ]:
print("Game 4:")
play_game()

In [ ]:
print("Game 5:")
play_game()